In [ ]:
import mapsims
import numpy as np
import healpy as hp
import pysm3.units as u
from pathlib import Path
%matplotlib inline
import matplotlib.pyplot as plt

# Access existing noise map based simulations

The last noise simulation, just released is [`202006_noise`](https://github.com/simonsobs/map_based_simulations/tree/master/202006_noise).

What is different in noise releases is that just 1 realization of full mission and splits is saved on disk, the others can be generated on-the-fly using `mapsims`, as we will see in the next notebook.

In [ ]:
channels = mapsims.parse_channels("tube:ST2")

In [ ]:
channel = channels[0][0]

In [ ]:
print(channel)

This release has variable $N_{side}$ based on the channel, see [this table](https://github.com/simonsobs/mapsims/blob/master/mapsims/data/so_default_resolution.csv), we have a utility function to retrieve that:

In [ ]:
nside = mapsims.get_default_so_resolution(channel)

# Load maps from a map based release at NERSC

In [ ]:
folder = Path("/project/projectdirs/sobs/v4_sims/mbs/")
release = "202006_noise"

In [ ]:
filename_template = "{content}/{num:04d}/simonsobs_{content}_uKCMB_{tube}_{band}_nside{nside}_{num:04d}_{split}_of_{nsplits}.fits"

In [ ]:
filenames = [Path(folder) / release / filename_template.format(nside=nside, content="noise", num=0, 
                                                   telescope=channel.telescope, tube=channel.tube, band=channel.band,
                                                             split=1, nsplits=1)]

In [ ]:
for split in range(1, 5):
    filenames += [Path(folder) / release / filename_template.format(nside=nside, content="noise", num=0, 
                                                   telescope=channel.telescope, tube=channel.tube, band=channel.band,
                                                             split=split, nsplits=4)]

In [ ]:
maps = [hp.read_map(filename, (0,1,2)) for filename in filenames]

In [ ]:
hp.mollview(maps[0][1], min=-20, max=20, title="EE map: " + channel.tag, unit="uK_CMB")

# Load the theoretical noise curve

Using `mapsims`, we can load the noise spectra generated from the `so_noise_models` package.
By default they are generated for the full sky, therefore we need to scale down by the sky fraction
to get the expected noise for a partial sky survey.

In [ ]:
noise = mapsims.SONoiseSimulator(nside=nside)

In [ ]:
ell, ps_T, ps_P = noise.get_fullsky_noise_spectra(tube=channel.tube)

In [ ]:
plt.loglog(ell, ps_P[0]);

# Load hitmaps, apodize and take spectra

Noise is very inhomogeneous, a direct anafast would be dominated by the border pixels with just a few hits, the trick is to multiply by the hitmap and then renormalize by the mean of the hitmap squared (as the $C_\ell$ are in power).

The sky fraction instead is needed because `anafast` always takes a transform of the full sky setting 0 on all the missing pixels.
Therefore the spectrum power is averaged down and we need to scale back by the sky fraction.

In [ ]:
hitmaps, ave_nhits = noise.get_hitmaps(tube=channel.tube)

In [ ]:
sky_fraction = (hitmaps[0] > 0).sum() / len(hitmaps[0])

In [ ]:
sky_fraction

In [ ]:
cls = [hp.anafast(m * hitmaps[0]) / np.mean(hitmaps[0]**2) / sky_fraction for m in maps]

In [ ]:
plt.figure(figsize=(8, 5))
plt.loglog(ell, ps_P[0] * sky_fraction)
plt.loglog(ell, ps_P[0] * sky_fraction * 4)

for cl in cls:
    plt.loglog(cl[1], alpha=.5)
plt.xlabel("ell")
plt.ylabel("C_ell [uK^2]");

# Add foregrounds and CMB

Currently the noise simulations are at variable $N_{side}$ while the foregrounds and CMB are 512/4096, so we might need to use `hp.ud_grade`. We are lucky that this channel is already at 512 so we do not need to.

The next foreground release will also be at variable $N_{side}$ so this issue will not apply anymore.

In [ ]:
folder = "/project/projectdirs/sobs/v4_sims/mbs/201906_highres_foregrounds_extragalactic_tophat"

In [ ]:
filename_template = "{nside}/{content}/{num:04d}/simonsobs_{content}_uKCMB_{telescope}{band}_nside{nside}_{num:04d}.fits"

In [ ]:
filename = Path(folder) / filename_template.format(nside=512, content="combined", num=0, 
                                                   telescope=channel.telescope.lower(), band=channel.band)

In [ ]:
foregrounds_cmb = hp.ma(hp.read_map(filename, (0,1,2)))

In [ ]:
maps = [each + foregrounds_cmb for each in maps]

In [ ]:
hp.mollview(maps[0][1], min=-20, max=20, title="EE map with foregrounds: " + channel.tag, unit="uK_CMB")